# Projet maison n° 4

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas, pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [47]:
# names us
def df_names_us():
    import re
    import glob
    files = glob.glob('names/*.txt')
    list_df = []
    for file in files:
        file_year = re.findall('\d+', file)
        temp_df = pd.read_csv(file, header = None, names = ['name', 'gender', 'births'])
        temp_df['year'] = file_year[0]
        list_df.append(temp_df) 
    df = pd.concat(list_df).sort_values('year').reset_index(drop=True)
    df = df[['year', 'name', 'gender', 'births']]
    return df

In [48]:
df_names_us()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Vern,M,19
2,1880,Stewart,M,19
3,1880,Randolph,M,19
4,1880,Lucien,M,19
...,...,...,...,...
1989396,2019,Dreama,F,9
1989397,2019,Dorthy,F,9
1989398,2019,Domino,F,9
1989399,2019,Elainna,F,9


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [39]:
# names fr
import numpy as np
def df_names_fr():
    df = pd.read_csv('nat2019.csv', sep=';')
    df = df.rename(columns={'sexe': 'gender', 'preusuel': 'name', 'annais': 'year', 'nombre': 'births'})
    df = df[['year', 'name', 'gender', 'births']]
    df['gender'] = df['gender'].map({1: 'M', 2: 'F'})
    df['name'] = df['name'].apply(str)
    df = df[(df['year'] != 'XXXX') & (df['name'].str.contains('_') == False)]
    df = df[df['name'].apply(lambda x: len(x) >= 2)]
    df['name'] = df['name'].apply(str.capitalize)
    df  = df.sort_values(['year', 'gender', 'births', 'name'], ascending=(True, True, False, True)).reset_index(drop=True)
    return df

In [40]:
df_names_fr()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
615907,2019,Éliel,M,3
615908,2019,Énes,M,3
615909,2019,Énoa,M,3
615910,2019,Éros,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [25]:
# taux de change
import re
import numpy as np
def df_taux_change(devises):
    df = pd.read_csv('Webstat_Export_20201031.csv', sep=';', decimal=',', skiprows=range(1,6))
    df = df.rename(columns = {'Titre :': 'Date'})
    df['Date'] = pd.to_datetime(df['Date'],format = '%d/%m/%Y')
    df = df.set_index('Date')
    column_names = df.columns.map(lambda x: re.search('\((.*)\)', x))
    df.columns = [x.group(1)[0:3] for x in column_names]
    df = df.applymap(lambda x: np.nan if x == '-' else x)
    df = df.applymap(lambda x: float(x.replace(',', '.')) if isinstance(x, str) else x)
    df = df.dropna()
    df = df[devises]
    
    return df

In [26]:
df_taux_change(['USD', 'GBP'])

,USD,GBP
Date,,
2006-12-29,1.3170,0.67150
2006-12-28,1.3173,0.67115
2006-12-27,1.3159,0.67085
2006-12-22,1.3192,0.67150
2006-12-21,1.3178,0.67160
...,...,...
2000-07-25,0.9410,0.62010
2000-07-24,0.9332,0.61510
2000-07-21,0.9363,0.61850


### Tests

In [27]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 1989401)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 615912)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [32]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [49]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 4.777s

OK
